In [1]:
import time
import datetime
import pandas as pd
from neo_finrl.env_fx_trading.env_fx import tgym

c:\github\NeoFinRL\.venv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [ ]:
import os
files=[]
for i in range(0,4): 
    file = f"./data/split/GBPUSD/weekly/GBPUSD_2017_{i}.csv"
    if os.path.isfile(file):
        files.append(file)
            
    else:
        print(f'not exist: {file}')
print(files)    


In [3]:
def train(env, agent, files, model_name, if_vix = True,**kwargs):
    learning_rate = kwargs.get('learning_rate', 3e-4)
    batch_size = kwargs.get('batch_size', 1024 )
    gamma = kwargs.get('gamma', 0.99)
    seed = kwargs.get('seed', 1)
    total_timesteps = kwargs.get('total_timesteps', 1e5)
    net_dimension = kwargs.get('net_dimension', 2**9)
    cwd = kwargs.get('cwd','./'+str(agent))

    # env_instance = map(env, [pd.read_csv(f) for f in files])
    if agent == 'ppo':
        from stable_baselines3 import PPO
        from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
        ppo_kwargs = {"learning_rate":learning_rate,
                        "n_steps":2048, 
                        "batch_size":batch_size, 
                        "ent_coef":0.0, 
                        "gamma":gamma, 
                        "seed":seed, 
                        "verbose":0, 
                        "epsilon":0.5,
                        "tensorboard_log":"./data/tensorboard/"}

        # env_train = [x.get_sb_env for x in env_instance ]
        vector_env = [lambda:env(df=pd.read_csv(f)) for f in files]
        env_train = SubprocVecEnv(vector_env)
        start_time = time.time()
        s = datetime.datetime.now()
        print(f'Training start: {s}')
        if os.path.exists(model_name):
            model = PPO.load(model_name, env=env_train, **ppo_kwargs)
            print(f'retaining model: {model_name}')
            model.learn(total_timesteps=total_timesteps)
            model.save(model_name)
        else:
            model = PPO("MlpPolicy", env=env_train, **ppo_kwargs)
            model.learn(total_timesteps=total_timesteps)
            model.save(model_name)

        print('Training finished!')
        model_name_session = "./data/models/GBPUSD-week-" + s.strftime('%Y%m%d%H%M%S')
        model.save(model_name_session)
        print(f'Trained model saved in {model_name_session}')
        print(f"trainning time: {(time.time() - start_time)}")

    else:
        raise ValueError('DRL library input is NOT supported. Please check.')
#'total_timesteps', 1e6
#trainning time: 2398.8598108291626

In [4]:
import os
m = './data/models/gbpusd-week.zip'
for i in range(0,2): 
    file = f"./data/split/GBPUSD/weekly/GBPUSD_2017_{i}.csv"
    if os.path.isfile(file):
        print(f'training on {file}')
        train(env=tgym,agent="ppo",model_name = m, files=[file])            
    else:
        print(f'not exist: {file}')
 

training on ./data/split/GBPUSD/weekly/GBPUSD_2017_0.csv
Training start: 2021-11-06 07:01:42.209447
retaining model: ./data/models/gbpusd-week.zip
Training finished!
Trained model saved in ./data/models/GBPUSD-week-20211106070142
trainning time: 155.5560007095337
training on ./data/split/GBPUSD/weekly/GBPUSD_2017_1.csv
Training start: 2021-11-06 07:04:28.135808
retaining model: ./data/models/gbpusd-week.zip
Training finished!
Trained model saved in ./data/models/GBPUSD-week-20211106070428
trainning time: 142.79399943351746


NameError: name 'files' is not defined

In [ ]:
run at 10:36 to check loading time.

In [ ]:
import pandas as pd
from neo_finrl.env_fx_trading.env_fx import tgym
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
file ="./data/split/GBPUSD/weekly/GBPUSD_2017_0.csv"
df = pd.read_csv(file)
t = tgym(df)

# if model: del model # remove to demonstrate saving and loading
model_name='./data/models/GBPUSD-week'
model = PPO.load(model_name)

start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    print(action, rewards)
    t.render(mode='graph')
print(f"--- running time: {(time.time() - start_time)}---")